In [ ]:
from google.colab import drive 
drive.mount('/content/drive',force_remount=True) 

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Web Mining")

In [ ]:
!pip install kmodes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install uszipcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade kmodes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from kmodes.kmodes import KModes
from tqdm import tqdm
from uszipcode import SearchEngine
%matplotlib inline

In [ ]:
## Load movies data
# movies = pd.read_csv('./movie_final_full_dataset.csv', encoding="ISO-8859-1", skipinitialspace=True)
ratings = pd.read_csv('./ml-1m/ratings.dat', sep="::", names=['UserID', 'MovieID', 'Rating', 'Timestamp'], skipinitialspace=True)
# users = pd.read_csv('ml-1m/users.dat', sep="::", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"],skipinitialspace=True) 

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
allusers= ratings['UserID']
print(len(allusers))
uniqueUsers = list(dict.fromkeys(allusers))
len(uniqueUsers)

1000209


6040

In [ ]:
user2movie={}
for user in uniqueUsers:
    user2movie[user]=set(ratings.loc[(ratings['UserID'] == user)]['MovieID'])

In [ ]:
moviesnumbers=[]
for user in uniqueUsers:
    dic={}
    dic["UserID"]=user
    dic["No. Rated Movies"]=len(user2movie[user])
    moviesnumbers.append(dic)
dfMovieNumber=pd.DataFrame(moviesnumbers).sort_values(by=['No. Rated Movies'], ascending=False)
dfMovieNumber=dfMovieNumber.set_index('UserID')

In [ ]:
# Function to split groups evenly
def splitList (lst, n):
    it = iter(lst)
    new = [[next(it) for _ in range(n)] for _ in range(len(lst) // n)]
    # print(new)
    for i, x in enumerate(it):
        new[i%(len(new))].append(x)
    # print(new)
    return new

In [ ]:
# # Elbow curve to find optimal K
# cost = []
# K = range(1,100)
# for num_clusters in tqdm(list(K)):
#     kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
#     kmode.fit_predict(dfc)
#     cost.append(kmode.cost_)
    
# plt.plot(K, cost, 'bx-')
# plt.xlabel('No. of clusters')
# plt.ylabel('Cost')
# plt.title('Elbow Method For Optimal k')
# plt.show()

In [ ]:
dfc=dfMovieNumber.copy()
# Building the model with 20 clusters
kmode = KModes(n_clusters=50, init = "random", n_init=10 , verbose=1, random_state=42)
clusters = kmode.fit_predict(dfc)
dfc.insert(0, "Cluster", clusters, True)
dfc.reset_index(level=0, inplace=True)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 102, cost: 4569.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 123, cost: 4699.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 203, cost: 4693.0
Run 3, iteration: 2/100, moves: 6, cost: 4693.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 84, cost: 4727.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 4762.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 6, iteration: 1/100, moves: 118, cost: 4476.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 7, iteration: 1/100, moves: 210, cost: 4838.0
Init: initializing centroids
Init: initializi

In [ ]:
# Splitting clusters into groups and combining list of groups into single list
groupsli = []
for i in dfc['Cluster'].unique():
  li2=dfc['UserID'].loc[dfc['Cluster']==i]
  # print(3,len(list(li2)))
  # print(i,(list(li2)))
  n=3
  if len(list(li2))>n:
    li3 = splitList(li2,n)
    groupsli.extend(li3)
test=dfc.copy()
test['Group']=0
# Allocating groups based on index of combined list
for j in groupsli:
  for k in j:
    test['Group'].loc[test['UserID']==k] = groupsli.index(j)+1
# test

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
i=0
groupIds=[]
for g in test['Group'].unique():
  df=test.loc[test['Group'] == g]
  common=user2movie[df['UserID'].iat[0]]
  for k in df['UserID']:
    common=common.intersection(user2movie[k])
  if len(common)>100:
    i+=1
    groupIds.append(g)
    # print("group ",g," has ",len(common))
print("no. of groups",i)

no. of groups 102


In [ ]:
# Exporting datset
groups=test[test['Group'].isin(groupIds)]
groups.to_csv("./Pipline/users_grouped_by3.csv", index=False)
groups

,UserID,Cluster,No. Rated Movies,Group
6,3618,0,1344,3
7,2063,0,1323,3
8,1150,0,1302,3
9,1015,0,1286,4
10,5795,0,1277,4
...,...,...,...,...
436,3892,0,477,143
437,2996,0,477,143
456,757,0,465,150
457,1842,0,465,150
